In [1]:
# 텐서플로우 2.0 기반의 huggingface 라이브러리 패키지 설치
!pip install transformers
!pip install sentencepiece

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import sentencepiece as spm

In [3]:

!wget https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all1.csv
!wget https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all.csv

--2021-06-25 05:54:35--  https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299412 (292K) [text/plain]
Saving to: ‘all1.csv.1’

all1.csv.1          100%[===================>] 292.39K  --.-KB/s    in 0.02s   

2021-06-25 05:54:35 (13.1 MB/s) - ‘all1.csv.1’ saved [299412/299412]

--2021-06-25 05:54:35--  https://raw.githubusercontent.com/yunakim2/OpenSourceProject/stock_report/data/labeled/all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

## **데이터 셋 train / test 데이터로 나눔**

In [4]:
train = pd.read_csv('all1.csv', encoding=  'utf8', sep=',')
train['label']=(train['label']>=0).astype(int)

test = pd.read_csv('all.csv', encoding= 'utf8', sep=',', error_bad_lines=False)
test['label']=(test['label']>=0).astype(int)


print('전체 데이터 개수 : ', (len(train) + len(test)))

print('train 데이터 개수 : ', len(train))
print('test 데이터 개수 : ', len(test))

전체 데이터 개수 :  2763
train 데이터 개수 :  2086
test 데이터 개수 :  677


In [5]:
test[:5]

,Unnamed: 0,Unnamed: 0.1,title,time,label,ChangeK
0,0,4,"종목명 to benefit from strong yen, pay high divid...",2016-06-29 15:26:05,1,0.010402
1,1,5,"[Hot-Line] “종목명, 외국인 VIP 증가에 실적 양호”",2016-07-19 08:12:04,1,-0.002088
2,2,6,"종목명, 2분기 영업이익 314억원…전년 동기比 44.8%↑",2016-08-05 17:29:04,1,0.006531
3,3,7,마카오 카지노경기 26개월만에 회복…파라다이스·종목명 `들썩`,2016-09-02 16:04:04,0,0.010680
4,4,8,"[Hot-Line] “종목명, 중국인 VIP 정체…내년 성장 동력 부재”",2016-11-07 08:45:07,0,0.007851


## **데이터 전처리**

- ↑, ↓ -> 상승 , 하락으로 변경

- [Hot-Line] 제거

- 그외 영어, 한글, 숫자 제외 나머지 문자열 제거

In [6]:
import re
def processing(data):
  data['title'] = data['title'].str.replace('Hot-Line','')
  data['title'] = data['title'].str.replace('↑',' 상승 ')
  data['title'] = data['title'].str.replace('↓', ' 하락 ')

  data['title'] = data['title'].str.replace('[^a-zA-Z0-9ㄱ-힗 ]', '')
  
  return data

In [7]:
test = processing(test)
train = processing(train)

test[:5]

,Unnamed: 0,Unnamed: 0.1,title,time,label,ChangeK
0,0,4,종목명 to benefit from strong yen pay high dividends,2016-06-29 15:26:05,1,0.010402
1,1,5,종목명 외국인 VIP 증가에 실적 양호,2016-07-19 08:12:04,1,-0.002088
2,2,6,종목명 2분기 영업이익 314억원전년 동기比 448 상승,2016-08-05 17:29:04,1,0.006531
3,3,7,마카오 카지노경기 26개월만에 회복파라다이스종목명 들썩,2016-09-02 16:04:04,0,0.010680
4,4,8,종목명 중국인 VIP 정체내년 성장 동력 부재,2016-11-07 08:45:07,0,0.007851


## **bert input 생성**
한글 데이터 분석하기 위해 한국어 데이터로 훈련된 koBERT 사용


https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드를 가져옴

In [8]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [10]:
# koBERT 토크나이즈 임포트
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

## **bertClassificationModel의 input으로 변환하기**

bertClassifiactionModel의 인풋으로 **토큰, 세그먼트, 마스크**으로 이루어져있어 test,train 문장들을 **토큰 세그먼트,마스크 인풋으로 변환하는 과정이 필요함**


### 1. tokenizer 예시

- `tokenizer.tokenize` => 문장을 토큰화
- `tokenizer.encode` => 문장을 버트 모델의 인풋 토큰값으로 변경

In [11]:
print(tokenizer.tokenize(test.iloc[0]['title']))

['▁종목', '명', '▁to', '▁', 'b', 'en', 'e', 'f', 'it', '▁', 'f', 'ro', 'm', '▁', 'st', 'ro', 'n', 'g', '▁', 'y', 'en', '▁', 'p', 'ay', '▁', 'h', 'i', 'g', 'h', '▁', 'd', 'i', 'v', 'id', 'en', 'd', 's']


In [12]:
print(tokenizer.encode(test.iloc[0]['title']))

[2, 4202, 6204, 709, 517, 380, 392, 389, 398, 413, 517, 398, 439, 423, 517, 441, 439, 425, 399, 517, 458, 392, 517, 432, 379, 517, 401, 405, 399, 401, 517, 388, 405, 453, 407, 392, 388, 440, 3]


### 1. 토큰 인풋 변환

문장 토크나이징 후 `tokenizer.encode()` 결과 값으로 문장이 `유효한 값`이면 **1**로, `유효하지 않으면` **0**으로 채워
문장 길이가 다르지만 **버트의 인풋길이를 일정하게 고정**하기 위해 버트에서 지정한 문장 길이를 초과하면 패딩값 0으로 채운다.


In [13]:
print(tokenizer.encode(test.iloc[0]['title'], max_length=64, pad_to_max_length=True))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 4202, 6204, 709, 517, 380, 392, 389, 398, 413, 517, 398, 439, 423, 517, 441, 439, 425, 399, 517, 458, 392, 517, 432, 379, 517, 401, 405, 399, 401, 517, 388, 405, 453, 407, 392, 388, 440, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


### 2. 세그멘트 인풋 변환

세그멘트는 bert 모형에 들어가 bert 모형에 맞게 고차원으로 임베딩 되는 원리임

버트 모형에서 문장이 앞 문장인지, 뒷 문장인지 표현해주는 것이고 
**지금 사용하고 있는 dataset은 한문장 이므로 '0'으로 통일한다.**

최대 길이를 128로 고정했으므로 세그먼트도 `[0]*128`가 되어야한다.


In [14]:
print([0]*128)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 3. 마스크 인풋

토큰 인풋에서 **패딩이 아닌 부분**은 `1` , **패딩인 부분**은 `0`으로 두게 되는 것

In [15]:
valid_num = len(tokenizer.encode(test.iloc[0]['title']))
print(valid_num * [1] + (128-valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## **버트 인풋으로 변환하는 함수 구현**

In [16]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 128 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df.iloc[i][DATA_COLUMN], truncation=True, padding='max_length', max_length=SEQ_LEN)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(상승 : 1 하락 0)을 targets 변수에 저장해 줌
        targets.append(data_df.iloc[i][LABEL_COLUMN])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 128
BATCH_SIZE = 32
# 뉴스 제목 칼럼
DATA_COLUMN = 'title'
# 상승인지 하락인지를 (1=상승,0=하락) 포함하고 있는 칼럼
LABEL_COLUMN = 'label'

**Train 데이터, Test 데이터 버트 인풋으로 변환**





In [17]:
train_x, train_y = load_data(train)
test_x, test_y = load_data(test)

100%|██████████| 677/677 [00:00<00:00, 1968.47it/s]


## **BertClassificatonModel 활용하여 감성분석 모델 구현**

In [18]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [19]:
bert_outputs

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model')>),
                              ('pooler_output',
                               <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)])

In [20]:
bert_outputs = bert_outputs[1]

In [21]:
# Rectified Adam 옵티마이저 사용
!pip install tensorflow_addons
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [22]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
news_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
news_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [23]:
news_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]            

In [24]:
news_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=64, validation_data=(test_x, test_y))

Epoch 1/4
33/33 [==============================] - 109s 2s/step - loss: 0.6888 - accuracy: 0.5594 - val_loss: 0.6954 - val_accuracy: 0.5199
Epoch 2/4
33/33 [==============================] - 58s 2s/step - loss: 0.6851 - accuracy: 0.5570 - val_loss: 0.6938 - val_accuracy: 0.5199
Epoch 3/4
33/33 [==============================] - 58s 2s/step - loss: 0.6856 - accuracy: 0.5599 - val_loss: 0.7012 - val_accuracy: 0.5199
Epoch 4/4
33/33 [==============================] - 58s 2s/step - loss: 0.6829 - accuracy: 0.5690 - val_loss: 0.7019 - val_accuracy: 0.5199


훈련 모델 예측 성능을 F1 - Score로 체크하기 위한 함수

In [25]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df.iloc[i]['title'], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [29]:
test_set = predict_load_data(test)
preds = news_model.predict(test_set)

100%|██████████| 677/677 [00:00<00:00, 2770.05it/s]


In [30]:
# 부정이면 0, 긍정이면 1 출력
preds

array([[0.5485114 ],
       [0.58797246],
       [0.58992994],
       [0.589275  ],
       [0.5888059 ],
       [0.5817048 ],
       [0.5824055 ],
       [0.58225584],
       [0.5883516 ],
       [0.5891346 ],
       [0.5810852 ],
       [0.58915377],
       [0.58798796],
       [0.5884642 ],
       [0.5890902 ],
       [0.58771974],
       [0.58796066],
       [0.5883722 ],
       [0.58857286],
       [0.58773863],
       [0.5898561 ],
       [0.5881874 ],
       [0.589619  ],
       [0.5892746 ],
       [0.5884478 ],
       [0.5887718 ],
       [0.5884131 ],
       [0.5868642 ],
       [0.5793407 ],
       [0.59027976],
       [0.5791807 ],
       [0.58195806],
       [0.58127016],
       [0.5875604 ],
       [0.58798   ],
       [0.5880241 ],
       [0.58955735],
       [0.58202434],
       [0.5883892 ],
       [0.58275515],
       [0.5885661 ],
       [0.5884396 ],
       [0.58186156],
       [0.5881049 ],
       [0.5892536 ],
       [0.5790975 ],
       [0.5812343 ],
       [0.588

 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [32]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       325
           1       0.52      1.00      0.68       352

    accuracy                           0.52       677
   macro avg       0.26      0.50      0.34       677
weighted avg       0.27      0.52      0.36       677



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
import logging
tf.get_logger().setLevel(logging.ERROR)